In [50]:
import requests
import os
import pandas as pd
import json
import sqlite3

In [52]:
# Provides better readability when parsing the raw Json files.
pd.set_option('display.max_columns', None)

In [13]:
API_Key = os.environ['POLYGON_API_KEY']

In [33]:
URL = 'https://api.polygon.io/vX/reference/financials'

headers = {
    'Authorization': 'Bearer %s' % API_Key
}
    
params = {
    'ticker': 'MSFT',
    'filing_date.gte': '2020-01-01',
    'timeframe': 'quarterly',
    'order': 'desc',
    'limit': 100,
    'sort': 'period_of_report_date'
}
    

response = requests.request(
    'GET', URL, params = params, headers = headers
)

print(response.status_code)
print(response.headers)

temp_Json = json.loads(
    response.text
)
    

200
{'Server': 'nginx/1.19.2', 'Date': 'Sun, 06 Aug 2023 20:38:46 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'X-Item-Count': '11', 'X-Request-Id': '79c9ce726b9e448d61b54af232f64f38', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains'}


In [56]:
temp_DF = pd.json_normalize(
    temp_Json['results'][0], max_level = 1
)
temp_DF.head()

,id,start_date,end_date,filing_date,acceptance_datetime,timeframe,fiscal_period,fiscal_year,cik,sic,tickers,company_name,source_filing_url,source_filing_file_url,financials.cash_flow_statement,financials.comprehensive_income,financials.income_statement,financials.balance_sheet
0,0000789019:2023:Q3,2023-01-01,2023-03-31,2023-04-25,2023-04-25T20:07:43Z,quarterly,Q3,2023,0000789019,7372,[MSFT],MICROSOFT CORP,https://api.polygon.io/v1/reference/sec/filing...,http://api.polygon.io/v1/reference/sec/filings...,{'net_cash_flow_from_investing_activities': {'...,{'comprehensive_income_loss': {'value': 191120...,"{'revenues': {'value': 52857000000.0, 'unit': ...","{'liabilities': {'value': 185405000000.0, 'uni..."


In [72]:
file_name = temp_Json['results'][0]['tickers'][0]

with open(f'{file_name}.json', 'w') as outfile:
    json.dump(temp_Json, outfile)

In [73]:
with open(f'{file_name}.json') as json_file:
    data = json.load(json_file)

In [74]:
data

{'results': [{'id': '0000789019:2023:Q3',
   'start_date': '2023-01-01',
   'end_date': '2023-03-31',
   'filing_date': '2023-04-25',
   'acceptance_datetime': '2023-04-25T20:07:43Z',
   'timeframe': 'quarterly',
   'fiscal_period': 'Q3',
   'fiscal_year': '2023',
   'cik': '0000789019',
   'sic': '7372',
   'tickers': ['MSFT'],
   'company_name': 'MICROSOFT CORP',
   'source_filing_url': 'https://api.polygon.io/v1/reference/sec/filings/0000950170-23-014423',
   'source_filing_file_url': 'http://api.polygon.io/v1/reference/sec/filings/0000950170-23-014423/files/msft-20230331_htm.xml',
   'financials': {'cash_flow_statement': {'net_cash_flow_from_investing_activities': {'value': -3264000000.0,
      'unit': 'USD',
      'label': 'Net Cash Flow From Investing Activities',
      'order': 400},
     'net_cash_flow_from_operating_activities': {'value': 24441000000.0,
      'unit': 'USD',
      'label': 'Net Cash Flow From Operating Activities',
      'order': 100},
     'net_cash_flow_from_